In [1]:
import os, sys
project_dir = os.path.join(os.getcwd(),'..')
if project_dir not in sys.path:
    sys.path.append(project_dir)

attention_dir = os.path.join(project_dir, 'modules/AttentionMap')
if attention_dir not in sys.path:
    sys.path.append(attention_dir)

sparse_dir = os.path.join(project_dir, 'modules/Sparse')
if sparse_dir not in sys.path:
    sys.path.append(sparse_dir) 

import torch
from torch import nn
from derma.architecture import InvertedResidual
from torchvision.models import MobileNetV2
from torchvision import datasets, transforms

In [2]:
input_test = torch.rand((1,3,64,64))

# Original setting for mobilenet v2 (https://github.com/pytorch/vision/blob/main/torchvision/models/mobilenetv2.py)
inverted_residual_setting = [
        # t, c, n, s
        [1, 16, 1, 1],
        [6, 24, 2, 2],
        [6, 32, 3, 2],
        [6, 64, 4, 2],
        [6, 96, 3, 1],
        [6, 160, 3, 2],
        [6, 320, 1, 1],
    ]

model = MobileNetV2(num_classes=100, inverted_residual_setting=inverted_residual_setting, block=InvertedResidual)

print('Features Output shape: {}'.format(model.features(input_test).shape))
print('Classifier Output shape: {}'.format(model(input_test).shape))

Features Output shape: torch.Size([1, 1280, 2, 2])
Classifier Output shape: torch.Size([1, 100])


# Dataset

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((128, 128))
])

dataset = datasets.CIFAR100('data', train=True, transform=transform, download=True)
loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

Files already downloaded and verified


# Training

In [4]:
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import numpy as np

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-6)
criterion = nn.CrossEntropyLoss()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device) 

tb_writer = SummaryWriter('log/MobilenetCoordAtt')
running_avg_accuracy = 0
step = 0

n_epoch = 10
epoch_iterator = tqdm(
    range(n_epoch),
    leave=True,
    unit="epoch",
    postfix={"tls": "%.4f" % 1},
)

loss_tr = []
total = 0
correct = 0
step=0
for epoch in epoch_iterator:
    for idx, (inputs, targets) in enumerate(loader):
        optimizer.zero_grad()

        inputs = inputs.to(device)
        targets = targets.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss_tr.append(loss.item())

        loss.backward()
        optimizer.step()

        # Accuracy
        predict = torch.argmax(outputs, 1)
        total += targets.size(0)
        correct += torch.eq(predict, targets).sum().double().item()

        if idx % 150 == 0:
            epoch_iterator.set_postfix(
                tls="%.4f" % np.mean(loss_tr)
            )
            
            accuracy = correct/total
            tb_writer.add_scalar('train/loss', np.mean(loss_tr), step)
            tb_writer.add_scalar('train/accuracy', accuracy, step)
            step+=1

            loss_tr = []
            total = 0
            correct = 0


100%|██████████| 10/10 [28:46<00:00, 172.69s/epoch, tls=1.3077]
